[![下载Notebook](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/r1.9/resource/_static/logo_notebook.png)](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/notebook/r1.9/tutorials/zh_cn/beginner/mindspore_transforms.ipynb)&emsp;[![下载样例代码](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/r1.9/resource/_static/logo_download_code.png)](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/notebook/r1.9/tutorials/zh_cn/beginner/mindspore_transforms.py)&emsp;[![查看源文件](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/r1.9/resource/_static/logo_source.png)](https://gitee.com/mindspore/docs/blob/r1.10/tutorials/source_zh_cn/beginner/transforms.ipynb)

[基本介绍](https://www.mindspore.cn/tutorials/zh-CN/r1.10/beginner/introduction.html) || [快速入门](https://www.mindspore.cn/tutorials/zh-CN/r1.10/beginner/quick_start.html) || [张量 Tensor](https://www.mindspore.cn/tutorials/zh-CN/r1.10/beginner/tensor.html) || [数据集 Dataset](https://www.mindspore.cn/tutorials/zh-CN/r1.10/beginner/dataset.html) || **数据变换 Transforms** || [网络构建](https://www.mindspore.cn/tutorials/zh-CN/r1.10/beginner/model.html) || [函数式自动微分](https://www.mindspore.cn/tutorials/zh-CN/r1.10/beginner/autograd.html) || [模型训练](https://www.mindspore.cn/tutorials/zh-CN/r1.10/beginner/train.html) || [保存与加载](https://www.mindspore.cn/tutorials/zh-CN/r1.10/beginner/save_load.html) || [推理与部署](https://www.mindspore.cn/tutorials/zh-CN/r1.10/beginner/infer.html)

# 数据变换 Transforms

通常情况下，直接加载的原始数据并不能直接送入神经网络进行训练，此时我们需要对其进行数据预处理。MindSpore提供不同种类的数据变换（Transforms），配合数据处理Pipeline来实现数据预处理。所有的Transforms均可通过`map`方法传入，实现对指定数据列的处理。

`mindspore.dataset`提供了面向图像、文本、音频等不同数据类型的Transforms，同时也支持使用Lambda函数。下面分别对其进行介绍。

In [19]:
import numpy as np
from PIL import Image
from download import download
from mindspore.dataset import transforms, vision, text
from mindspore.dataset import GeneratorDataset, MnistDataset

## Common Transforms

`mindspore.dataset.transforms`模块支持一系列通用Transforms。这里我们以`Compose`为例，介绍其使用方式。

### Compose

`Compose`接收一个数据增强操作序列，然后将其组合成单个数据增强操作。我们仍基于Mnist数据集呈现Transforms的应用效果。

In [20]:
# Download data from open datasets

url = "https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/" \
      "notebook/datasets/MNIST_Data.zip"
path = download(url, "./", kind="zip", replace=True)

train_dataset = MnistDataset('MNIST_Data/train')


file_sizes: 100%|██████████████████████████| 10.8M/10.8M [00:01<00:00, 5.57MB/s]
Extracting zip file...
Successfully downloaded / unzipped to ./


In [21]:
image, label = next(train_dataset.create_tuple_iterator())
print(image.shape)

(28, 28, 1)


In [22]:
composed = transforms.Compose(
    [
        vision.Rescale(1.0 / 255.0, 0),
        vision.Normalize(mean=(0.1307,), std=(0.3081,)),
        vision.HWC2CHW()
    ]
)

In [23]:
train_dataset = train_dataset.map(composed, 'image')
image, label = next(train_dataset.create_tuple_iterator())
print(image.shape)

(1, 28, 28)


更多通用Transforms详见[mindspore.dataset.transforms](https://www.mindspore.cn/docs/zh-CN/r1.10/api_python/mindspore.dataset.transforms.html)。

## Vision Transforms

`mindspore.dataset.vision`模块提供一系列针对图像数据的Transforms。在Mnist数据处理过程中，使用了`Rescale`、`Normalize`和`HWC2CHW`变换。下面对其进行详述。

### Rescale

`Rescale`变换用于调整图像像素值的大小，包括两个参数：

- rescale：缩放因子。
- shift：平移因子。

图像的每个像素将根据这两个参数进行调整，输出的像素值为$output_{i} = input_{i} * rescale + shift$。

这里我们先使用numpy随机生成一个像素值在\[0, 255\]的图像，将其像素值进行缩放。

In [24]:
random_np = np.random.randint(0, 255, (48, 48), np.uint8)
random_image = Image.fromarray(random_np)
print(random_np)

[[ 59  38 206 ... 126 244 226]
 [ 27 113 135 ... 248   3   0]
 [106  13 154 ... 149   7 126]
 ...
 [142 135 222 ... 253  58 228]
 [110 239 114 ...  75 142  65]
 [  0 108 141 ... 145 159  11]]


为了更直观地呈现Transform前后的数据对比，我们使用Transforms的[Eager模式](https://www.mindspore.cn/tutorials/experts/zh-CN/r1.10/dataset/eager.html)进行演示。首先实例化Transform对象，然后调用对象进行数据处理。

In [25]:
rescale = vision.Rescale(1.0 / 255.0, 0)
rescaled_image = rescale(random_image)
print(rescaled_image)

[[0.23137257 0.14901961 0.8078432  ... 0.49411768 0.9568628  0.8862746 ]
 [0.10588236 0.4431373  0.5294118  ... 0.9725491  0.01176471 0.        ]
 [0.4156863  0.0509804  0.6039216  ... 0.58431375 0.02745098 0.49411768]
 ...
 [0.5568628  0.5294118  0.8705883  ... 0.9921569  0.227451   0.8941177 ]
 [0.43137258 0.93725497 0.44705886 ... 0.29411766 0.5568628  0.25490198]
 [0.         0.42352945 0.5529412  ... 0.5686275  0.62352943 0.04313726]]


可以看到，使用`Rescale`后的每个像素值都进行了缩放。

### Normalize

`Normalize`变换用于对输入图像的归一化，包括三个参数：

- mean：图像每个通道的均值。
- std：图像每个通道的标准差。
- is_hwc：输入图像格式为(height, width, channel)还是(channel, height, width)。

图像的每个通道将根据`mean`和`std`进行调整，计算公式为$output_{c} = (input_{c} - \frac{mean_{c}}{std_{c}})$，其中 $c$代表通道索引。

In [26]:
normalize = vision.Normalize(mean=(0.1307,), std=(0.3081,))
normalized_image = normalize(rescaled_image)
print(normalized_image)

[[ 0.32675287  0.05945994  2.1978035  ...  1.1795447   2.6814764
   2.452368  ]
 [-0.08055063  1.0140777   1.2940987  ...  2.7323892  -0.38602826
  -0.42421296]
 [ 0.92498    -0.2587459   1.5359352  ...  1.472294   -0.33511534
   1.1795447 ]
 ...
 [ 1.3831964   1.2940987   2.4014552  ...  2.7960305   0.31402466
   2.4778247 ]
 [ 0.9758929   2.617835    1.0268059  ...  0.5304046   1.3831964
   0.40312228]
 [-0.42421296  0.9504364   1.3704681  ...  1.4213811   1.5995764
  -0.2842024 ]]


### HWC2CWH

`HWC2CWH`变换用于转换图像格式。在不同的硬件设备中可能会对(height, width, channel)或(channel, height, width)两种不同格式有针对性优化。MindSpore设置HWC为默认图像格式，在有CWH格式需求时，可使用该变换进行处理。

这里我们先将前文中`normalized_image`处理为HWC格式，然后进行转换。可以看到转换前后的shape发生了变化。

In [27]:
hwc_image = np.expand_dims(normalized_image, -1)
hwc2cwh = vision.HWC2CHW()
chw_image = hwc2cwh(hwc_image)
print(hwc_image.shape, chw_image.shape)

(48, 48, 1) (1, 48, 48)


更多Vision Transforms详见[mindspore.dataset.vision](https://www.mindspore.cn/docs/zh-CN/r1.10/api_python/mindspore.dataset.vision.html)。

## Text Transforms

`mindspore.dataset.text`模块提供一系列针对文本数据的Transforms。与图像数据不同，文本数据需要有分词（Tokenize）、构建词表、Token转Index等操作。这里简单介绍其使用方法。

首先我们定义三段文本，作为待处理的数据，并使用`GeneratorDataset`进行加载。

In [28]:
texts = [
    'Welcome to Beijing',
    '北京欢迎您！',
    '我喜欢China!',
]

In [29]:
test_dataset = GeneratorDataset(texts, 'text')

### BasicTokenizer

分词（Tokenize）操作是文本数据的基础处理方法，MindSpore提供多种不同的Tokenizer。这里我们选择基础的`BasicTokenizer`举例。配合`map`，将三段文本进行分词，可以看到处理后的数据成功分词。

In [30]:
test_dataset = test_dataset.map(text.BasicTokenizer())
print(next(test_dataset.create_tuple_iterator()))

[Tensor(shape=[5], dtype=String, value= ['我', '喜', '欢', 'China', '!'])]


### Lookup

`Lookup`为词表映射变换，用来将Token转换为Index。在使用`Lookup`前，需要构造词表，一般可以加载已有的词表，或使用`Vocab`生成词表。这里我们选择使用`Vocab.from_dataset`方法从数据集中生成词表。

In [31]:
vocab = text.Vocab.from_dataset(test_dataset)

获得词表后我们可以使用`vocab`方法查看词表。

In [32]:
print(vocab.vocab())

{'迎': 11, '我': 10, '您': 9, '京': 6, 'to': 5, '！': 12, '喜': 8, 'Welcome': 4, 'China': 3, '北': 7, 'Beijing': 2, '!': 1, '欢': 0}


生成词表后，可以配合`map`方法进行词表映射变换，将Token转为Index。

In [33]:
test_dataset = test_dataset.map(text.Lookup(vocab))
print(next(test_dataset.create_tuple_iterator()))

[Tensor(shape=[3], dtype=Int32, value= [4, 5, 2])]


更多Text Transforms详见[mindspore.dataset.text](https://www.mindspore.cn/docs/zh-CN/r1.10/api_python/mindspore.dataset.text.html)。

## Lambda Transforms

Lambda函数是一种不需要名字、由一个单独表达式组成的匿名函数，表达式会在调用时被求值。Lambda Transforms可以加载任意定义的Lambda函数，提供足够的灵活度。在这里，我们首先使用一个简单的Lambda函数，对输入数据乘2：

In [34]:
test_dataset = GeneratorDataset([1, 2, 3], 'data', shuffle=False)
test_dataset = test_dataset.map(lambda x: x * 2)
print(list(test_dataset.create_tuple_iterator()))

[[Tensor(shape=[], dtype=Int64, value= 2)], [Tensor(shape=[], dtype=Int64, value= 4)], [Tensor(shape=[], dtype=Int64, value= 6)]]


可以看到`map`传入Lambda函数后，迭代获得数据进行了乘2操作。

我们也可以定义较复杂的函数，配合Lambda函数实现复杂数据处理：

In [35]:
def func(x):
    return x * x + 2

test_dataset = test_dataset.map(lambda x: func(x))

In [36]:
print(list(test_dataset.create_tuple_iterator()))

[[Tensor(shape=[], dtype=Int64, value= 6)], [Tensor(shape=[], dtype=Int64, value= 18)], [Tensor(shape=[], dtype=Int64, value= 38)]]
